In [3]:
from pprint import pprint
import re

In [4]:
with open("./stored_authors/author_url_pairings.txt", encoding="utf8") as pairings_file:
    author_url_pairings = eval(pairings_file.read())

In [5]:
with open("./stored_authors/unknown_authors.txt") as unknowns_file:
    unknown_authors = eval(unknowns_file.read())

In [6]:
with open("./stored_authors/duplicate_authors.txt") as duplicates_file:
    duplicate_authors = eval(duplicates_file.read())

In [7]:
# Used for reference when checking for authors in duplicate list

unpacked_duplicates = set()
for author_list in duplicate_authors.values():
    for author in author_list:
        unpacked_duplicates.add(author)
        
sorted_unpacked_duplicates = sorted(list(unpacked_duplicates))


In [8]:
# pprint(sorted_unpacked_duplicates)

In [9]:
unknown_pairing = {}

for unknown in unknown_authors:
    unknown_pairing[unknown] = None

# pair_scraped_unknown(unknown_authors, unknown_pairing)

h_likely = [] # Highly likely, unknowns that have matches
possible = [] # Moderately likely, the ones whose possible matches have similar characters
unlikely = [] # Unlikely, few matching characters

for author, pair in unknown_authors.items():

    if author in author_url_pairings:
        author_url = author_url_pairings[author]
    else:
        author_url = "no"

#     print("-----new iteration-----")
#     print(author)
#     print(author_url) # 
#     print("--end of author url--")

    found = "Unsure"
    for pair_id in pair:
        pair_url = author_url_pairings[pair_id]

#         print(pair_url) #

        if "contributions" in author_url and "contributions" in pair_url:
            author_num = re.search(r"\d+", author_url).group(0)
            pair_num = re.search(r"\d+", pair_url).group(0)
            if author_num == pair_num:
                found = pair_id
                h_likely.append([author, pair_id])

#                 print("yes!") #

        elif author_url ==  pair_url:
            found = pair_id

#             print("yes!") #

    unknown_pairing[author] = found

In [11]:
pprint(h_likely)

[['0d853b61e5fab5c307961edfe4161eac73936c04',
  '5fad00e089988f39a9e20dd283cd2d1963b1dbb9'],
 ['1f5e77fb6eae60e5697b47a51a28135a54218fe3',
  'bcdaffee924e8a8f5c4f85576b0be66c4c621603'],
 ['261d4cefe788ce8950675624f76fe3b0bf2f42bb',
  'a443e241a9d1dfa6bf34643f71be2c8bab1ea12e'],
 ['2ad7d292f106360de8d0d3374b38249d3e1fabcb',
  '66bd6e2a22a84008b43ca6f41252251564e1b3a2'],
 ['34920abb810793267ab8f76e801feacff38f270f',
  '926d407a555591144571817f55d07c14ffa24f13'],
 ['440336c9743eb61828c50fd9976605b7f825dd34',
  '6bc95e425f0ac28caa0a4a7a7992cb8225f4289b'],
 ['49d55b33ee29beb4758fd5e569504db29ac77a27',
  'df4b20c3081113f1e2b6dac1dd019107e4a9248e'],
 ['4ed09a2248ccb222088792dc27adbfefed14570e',
  '06da5287c7fe7be0d6b133f6bfc0ee21d2dcc417'],
 ['52db2c7849175bcef1a5ca2e8da61e212b2109b3',
  'fa0f65d6ee08be2b510169c5738b78f4be538742'],
 ['58bab844f4d1e2f62440d5c98711590c8c6c1ec1',
  '4bfc4f702709467c98a9a83776f214a668345943'],
 ['5d59ab4c8a8b18af0136d539e397cf744e06e577',
  'd40adc7e6b5d0ee961dd9

In [234]:
duplicate_pairing = {}

for author, url in author_url_pairings.items():
    if author in sorted_unpacked_duplicates:
        duplicate_pairing[url] = []
        
# pair_scraped_duplicate(duplicate_authors, duplicate_pairing)

for author, pair in duplicate_authors.items():
    for pair_id in pair:
        duplicate_pairing[author_url_pairings[pair_id]].append(pair_id)

In [1]:
# pprint(unknown_pairing)

In [221]:
# pprint(duplicate_pairing)